# LDA - RF Pipeline 

## Import necessary packages

In [1]:
import time
import optuna
import numpy as np
from tqdm import tqdm
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import KFold, train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from bitermplus import BTM

from datasets import load_dataset

## Constants

In [2]:
RANDOM_SEED = 0

## Load the Hate Speech Filipino dataset from Hugging Face

In [3]:
dataset = load_dataset("hate_speech_filipino")

In [4]:
train_set = dataset["train"]
validation_set = dataset["validation"]
test_set = dataset["test"]

X_train, Y_train = train_set["text"], train_set["label"]
X_val, Y_val = validation_set["text"], validation_set["label"]
X_test, Y_test = test_set["text"], test_set["label"]

X = X_train + X_val + X_test
Y = Y_train + Y_val + Y_test

In [5]:
# Re-split the dataset into training, validation, and test sets
X_train, X_temp, Y_train, Y_temp = train_test_split(X, Y, test_size=0.2, random_state=RANDOM_SEED)
X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=0.5, random_state=RANDOM_SEED)

## Vectorize the texts to be able to perform LDA

In [6]:
# Define the CountVectorizer
vectorizer = CountVectorizer(max_df=0.95, min_df=2)
X_train_vector = vectorizer.fit_transform(X_train)

## Perform Latent Dirichlet Allocation on the training set

In [7]:
N_TOPICS = 10
print(f"Performing Latent Dirichlet Allocation for {N_TOPICS} topics")
lda = LatentDirichletAllocation(n_components=N_TOPICS, random_state=RANDOM_SEED)
X_train_topics = lda.fit_transform(X_train_vector)
print(f"Done performing Latent Dirichlet Allocation for {N_TOPICS} topics")

Performing Latent Dirichlet Allocation for 10 topics
Done performing Latent Dirichlet Allocation for 10 topics


In [8]:
# Transform validation and test data using the fitted vectorizer and LDA
X_val_counts = vectorizer.transform(X_val)
X_val_topics = lda.transform(X_val_counts)

X_test_counts = vectorizer.transform(X_test)
X_test_topics = lda.transform(X_test_counts)

## Search for the best hyperparameters of the Random Forest model

In [13]:
# Define the objective function for Optuna
def create_objective(X_train, Y_train, X_test, Y_test):
    def objective(trial):
        n_estimators = trial.suggest_int("n_estimators", 100, 1000)
        rf = RandomForestClassifier(n_estimators=n_estimators, random_state=RANDOM_SEED)
        rf.fit(X_train, Y_train)
        Y_pred = rf.predict(X_test)
        score = accuracy_score(Y_test, Y_pred)
        return score
    return objective

In [14]:
# Create a study object
study = optuna.create_study(direction="maximize", study_name=f"LDA_RF_Pipeline")

# Create the study objective
objective = create_objective(X_train_topics, Y_train, X_val_topics, Y_val)

# Execute an optimization
study.optimize(objective, n_trials=20, n_jobs=-1)

# Print the best trial results
print(f"Best Accuracy: {study.best_trial.value}")
print(f"Best hyperparameters: {study.best_trial.params}")

[I 2024-04-29 23:41:09,972] A new study created in memory with name: LDA_RF_Pipeline
[I 2024-04-29 23:41:24,394] Trial 1 finished with value: 0.8497730086669418 and parameters: {'n_estimators': 199}. Best is trial 1 with value: 0.8497730086669418.
[I 2024-04-29 23:41:32,720] Trial 10 finished with value: 0.8493602971522906 and parameters: {'n_estimators': 302}. Best is trial 1 with value: 0.8497730086669418.
[I 2024-04-29 23:41:38,811] Trial 2 finished with value: 0.8497730086669418 and parameters: {'n_estimators': 373}. Best is trial 1 with value: 0.8497730086669418.
[I 2024-04-29 23:41:53,524] Trial 4 finished with value: 0.8514238547255468 and parameters: {'n_estimators': 514}. Best is trial 4 with value: 0.8514238547255468.
[I 2024-04-29 23:42:04,880] Trial 7 finished with value: 0.8489475856376393 and parameters: {'n_estimators': 616}. Best is trial 4 with value: 0.8514238547255468.
[I 2024-04-29 23:42:10,452] Trial 5 finished with value: 0.8472967395790343 and parameters: {'n_est

Best Accuracy: 0.8514238547255468
Best hyperparameters: {'n_estimators': 514}


In [17]:
best_n_estimators = study.best_trial.params["n_estimators"]

rf = RandomForestClassifier(n_estimators=best_n_estimators, random_state=RANDOM_SEED)
rf.fit(X_train_topics, Y_train)
Y_pred = rf.predict(X_test_topics)
score = accuracy_score(Y_test, Y_pred)
print(f"Best LDA-RF Model Accuracy: {score}")

Best LDA-RF Model Accuracy: 0.8696369636963697
